In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')
path = "/content/drive/My Drive/GEARS"

os.chdir(path)

## 1. 通用的数据

In [ ]:
# Really need these
import pandas as pd 
import numpy as np
from numpy import *

# Handy for debugging
import gc
import time
import warnings
import os

# Date stuff
from datetime import datetime
from datetime import timedelta

# Nice graphing tools
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.offline as py
import plotly.tools as tls
import plotly.graph_objs as go
import plotly.tools as tls

# Machine learning tools
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix

# Performance measures
from sklearn.metrics import mean_squared_error

In [ ]:
# 读取 NRWQ database 数据
df_nrwq = pd.read_excel('NRWQ database 12_02_19.xlsx')

# 选出chl的数据，加上必要的日期、深度等信息
chl_col = ['Date', 'Season', 'Station', 'Depth', 'YSI_Temp', 'YSI_Salinity', 'YSI_DO', 'YSI_pH', 'POC']
chl_col = chl_col + ['NO3/NO2', 'NH4', 'PO4', 'TotalChla']

# 选出我们需要的数据
df_chla = df_nrwq[chl_col]
df_chla.rename(columns={'NO3/NO2':'NOx'}, inplace=True)

# 删除第一行数据
df_chla = df_chla.drop([0]) 

df_chla.columns

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning:

Unknown extension is not supported and will be removed

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['Date', 'Season', 'Station', 'Depth', 'YSI_Temp', 'YSI_Salinity',
       'YSI_DO', 'YSI_pH', 'POC', 'NOx', 'NH4', 'PO4', 'TotalChla'],
      dtype='object')

In [ ]:
# 将小于0的数据都填为0
df_chla['YSI_Temp'][df_chla.YSI_Temp < 0 ] = np.nan
df_chla['YSI_Salinity'][df_chla.YSI_Salinity < 0 ] = np.nan
df_chla['YSI_DO'][df_chla.YSI_DO < 0 ] = np.nan
df_chla['YSI_pH'][df_chla.YSI_pH < 0 ] = np.nan
df_chla['POC'][df_chla.POC < 0 ] = np.nan
df_chla['NOx'][df_chla.NOx < 0 ] = np.nan
df_chla['NH4'][df_chla.NH4 < 0 ] = np.nan

df_chla['PO4'] = pd.to_numeric(df_chla['PO4'] , errors='coerce')
df_chla['PO4'][df_chla.PO4 < 0 ] = np.nan

df_chla['TotalChla'][df_chla.TotalChla < 0 ] = np.nan

df_chla

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying t

,Date,Season,Station,Depth,YSI_Temp,YSI_Salinity,YSI_DO,YSI_pH,POC,NOx,NH4,PO4,TotalChla
1,1994-01-24 00:00:00,Winter,0.0,1.0,4.300000,0.000000,11.10,NaN,984.7,NaN,120.929,40.418,0.909647
2,1994-01-24 00:00:00,Winter,0.0,2.0,4.000000,0.000000,11.40,NaN,992.5,722.95,130.962,40.250,0.907074
3,1994-01-24 00:00:00,Winter,10.0,1.0,4.000000,0.000000,11.20,NaN,897.1,722.43,118.444,38.399,0.918636
4,1994-01-24 00:00:00,Winter,10.0,2.0,3.900000,0.000000,11.20,NaN,1026.8,773.65,129.121,38.399,1.117271
5,1994-01-24 00:00:00,Winter,20.0,1.0,3.000000,0.000000,11.50,NaN,683.8,674.90,124.887,37.558,1.183448
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12658,2019-07-09 00:00:00,Summer,140.0,2.0,28.279999,24.869999,0.71,7.500000,NaN,NaN,NaN,NaN,NaN
12659,2019-07-09 00:00:00,Summer,160.0,1.0,28.689999,16.410000,6.93,8.480000,NaN,NaN,NaN,NaN,NaN
12660,2019-07-09 00:00:00,Summer,160.0,2.0,28.269999,27.309999,0.86,7.550000,NaN,NaN,NaN,NaN,NaN
12661,2019-07-09 00:00:00,Summer,180.0,1.0,28.619999,17.900000,6.44,8.419999,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 并没有冗余的数据，每次采样都只记录一次，判断代码放到save中了
# 把depth=1或2留下，丢弃11D这种
df_chla = df_chla[ (df_chla['Depth'] == 1) | (df_chla['Depth'] == 2) ]

# 处理时间
df_chla['Date'] = [datetime.strptime(date, '%Y-%m-%d %H:%M:%S').date() for date in df_chla['Date'].astype(str).values.tolist()]

# 增加时间列
df_chla['Year'] = [datetime.strptime(date, '%Y-%m-%d').year for date in df_chla['Date'].astype(str).values.tolist()]
df_chla['Month'] = [datetime.strptime(date, '%Y-%m-%d').month for date in df_chla['Date'].astype(str).values.tolist()]
df_chla['Day'] = [datetime.strptime(date, '%Y-%m-%d').day for date in df_chla['Date'].astype(str).values.tolist()]

# 调整顺序
order = ['Date', 'Year', 'Month', 'Day', 'Season', 'Station', 'Depth', 'YSI_Temp', 'YSI_Salinity',
         'YSI_DO', 'YSI_pH', 'POC', 'NOx', 'NH4', 'PO4', 'TotalChla']
df_chla = df_chla[order]

df_chla.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

,Date,Year,Month,Day,Season,Station,Depth,YSI_Temp,YSI_Salinity,YSI_DO,YSI_pH,POC,NOx,NH4,PO4,TotalChla
1,1994-01-24,1994,1,24,Winter,0.0,1.0,4.3,0.0,11.1,NaN,984.7,NaN,120.929,40.418,0.909647
2,1994-01-24,1994,1,24,Winter,0.0,2.0,4.0,0.0,11.4,NaN,992.5,722.95,130.962,40.250,0.907074
3,1994-01-24,1994,1,24,Winter,10.0,1.0,4.0,0.0,11.2,NaN,897.1,722.43,118.444,38.399,0.918636
4,1994-01-24,1994,1,24,Winter,10.0,2.0,3.9,0.0,11.2,NaN,1026.8,773.65,129.121,38.399,1.117271
5,1994-01-24,1994,1,24,Winter,20.0,1.0,3.0,0.0,11.5,NaN,683.8,674.90,124.887,37.558,1.183448


In [ ]:
# 去除掉几个另类
df_chla = df_chla[ (df_chla['Station'] != 'Union Point Neuse') & (df_chla['Station'] != 'Streets Ferry Bridge')
            & (df_chla['Station'] != 'Union Point Trent') & (df_chla['Station'] != 'Camp Seafarer')
              & (df_chla['Station'] != 'Fisher Landing') & (df_chla['Station'] != 'Neuse River at Kinston')
                & (df_chla['Station'] != 95) & (df_chla['Station'] != 96) & (df_chla['Station'] != 80)
                  & (df_chla['Station'] != 130) & (df_chla['Station'] != 10) & (df_chla['Station'] != 165) ]

df_chla['Station'].value_counts()

30.0     1195
70.0     1195
120.0    1190
20.0     1162
0.0      1160
50.0     1085
60.0     1084
100.0    1020
140.0     999
160.0     997
180.0     809
Name: Station, dtype: int64

In [ ]:
# 选取1999-2018年
df_chla = df_chla[ (df_chla['Year'] >= 1999) & (df_chla['Year'] < 2019) ]

# 从新建立索引
df_chla = df_chla.reset_index(drop=True)
df_chla

,Date,Year,Month,Day,Season,Station,Depth,YSI_Temp,YSI_Salinity,YSI_DO,YSI_pH,POC,NOx,NH4,PO4,TotalChla
0,1999-01-05,1999,1,5,Winter,0.0,1.0,6.40,0.00,10.57,8.06,1070.700000,1099.89,53.44,16.22,0.219479
1,1999-01-05,1999,1,5,Winter,0.0,2.0,6.43,0.10,9.96,7.77,1141.666667,1108.73,58.17,20.92,0.798709
2,1999-01-05,1999,1,5,Winter,20.0,2.0,6.71,0.10,9.94,7.94,1281.066667,1120.90,56.45,22.18,1.210042
3,1999-01-05,1999,1,5,Winter,20.0,1.0,6.64,0.10,10.74,8.24,1076.300000,1114.91,52.17,19.00,1.002726
4,1999-01-05,1999,1,5,Winter,30.0,1.0,6.84,0.50,10.65,8.00,750.333333,930.14,82.34,19.32,0.739095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9889,2018-12-12,2018,12,12,Winter,140.0,2.0,9.00,7.34,10.87,7.67,1197.400000,199.00,32.40,11.80,6.779215
9890,2018-12-12,2018,12,12,Winter,160.0,1.0,8.61,6.86,11.17,7.97,1106.500000,220.00,36.60,14.00,8.684863
9891,2018-12-12,2018,12,12,Winter,160.0,2.0,8.87,10.33,10.51,7.79,1943.250000,70.60,45.30,5.87,9.031465
9892,2018-12-12,2018,12,12,Winter,180.0,1.0,8.55,9.91,11.07,8.00,1132.900000,84.10,42.30,7.46,7.145236


In [ ]:
# 填充缺失的数据
def interpolation(df_new, column_name):
  # 缺少测量值的站点名单
  df_blank = df_new[df_new[column_name].isnull()]
  df_fill = df_new.dropna(axis=0)
  station_List = df_blank['Station'].unique()
  # print(df_blank)

  for station in station_List:
    # 选出对应的station缺失的部分，以及用于填充的部分
    df_station_blank = df_blank[(df_blank['Station'] == station)]
    df_station_fill = df_fill[(df_fill['Station'] == station)]

    month_List = df_station_blank['Month'].unique()
    # print(df_station_blank)

    for month in month_List:
      # 从对应的站点中选出一个月
      df_month_blank = df_station_blank[(df_station_blank['Month'] == month)]
      df_month_fill = df_station_fill[(df_station_fill['Month'] == month)]

      depth_List = df_month_blank['Depth'].unique()
      # print(df_month_blank)

      for depth in depth_List:
        # 选出对应深度的来填充
        df_depth_blank = df_month_blank[(df_month_blank['Depth'] == depth)]
        df_depth_fill = df_month_fill[(df_month_fill['Depth'] == depth)]

        # 计算平均值来填充
        df_depth_blank[column_name] = np.mean(df_depth_fill[column_name])

        # 添加到总的数据中
        # print(df_depth_blank)
        df_fill = df_fill.append(df_depth_blank)

  return df_fill

In [ ]:
# 分类处理数据
df_ysi_temp = df_chla.groupby(['Date', 'Year', 'Month', 'Day', 'Season', 'Station', 'Depth'], as_index=False)['YSI_Temp'].mean()
df_ysi_salinity = df_chla.groupby(['Date', 'Year', 'Month', 'Day', 'Season', 'Station', 'Depth'], as_index=False)['YSI_Salinity'].mean()

df_YSI_Temp = interpolation(df_ysi_temp, 'YSI_Temp')
df_YSI_Salinity = interpolation(df_ysi_salinity, 'YSI_Salinity')

# 合并
df_chla_merge = pd.merge(df_YSI_Temp, df_YSI_Salinity, on=['Date', 'Year', 'Month', 'Day', 'Season', 'Station', 'Depth'], how='inner')

# 循环遍历处理
columns = ['YSI_DO', 'YSI_pH', 'POC', 'NOx', 'NH4', 'PO4', 'TotalChla']
for col in columns:
  df_temp = df_chla.groupby(['Date', 'Year', 'Month', 'Day', 'Season', 'Station', 'Depth'], as_index=False)[col].mean()
  df_inter_temp = interpolation(df_temp, col)
  df_chla_merge = pd.merge(df_chla_merge, df_inter_temp, on=['Date', 'Year', 'Month', 'Day', 'Season', 'Station', 'Depth'], how='inner')

# 排序整理
df_chla_merge.sort_values(by=['Year', 'Month', 'Day', 'Station'], axis=0, ascending=True, inplace=True)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
print(df_chla_merge.isnull().any())

df_chla_merge

Date            False
Year            False
Month           False
Day             False
Season          False
Station         False
Depth           False
YSI_Temp        False
YSI_Salinity    False
YSI_DO          False
YSI_pH          False
POC             False
NOx             False
NH4             False
PO4             False
TotalChla       False
dtype: bool


,Date,Year,Month,Day,Season,Station,Depth,YSI_Temp,YSI_Salinity,YSI_DO,YSI_pH,POC,NOx,NH4,PO4,TotalChla
0,1999-01-05,1999,1,5,Winter,0.0,1.0,6.40,0.00,10.57,8.06,1070.700000,1099.89,53.44,16.22,0.219479
1,1999-01-05,1999,1,5,Winter,0.0,2.0,6.43,0.10,9.96,7.77,1141.666667,1108.73,58.17,20.92,0.798709
2,1999-01-05,1999,1,5,Winter,20.0,1.0,6.64,0.10,10.74,8.24,1076.300000,1114.91,52.17,19.00,1.002726
3,1999-01-05,1999,1,5,Winter,20.0,2.0,6.71,0.10,9.94,7.94,1281.066667,1120.90,56.45,22.18,1.210042
4,1999-01-05,1999,1,5,Winter,30.0,1.0,6.84,0.50,10.65,8.00,750.333333,930.14,82.34,19.32,0.739095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9837,2018-12-12,2018,12,12,Winter,140.0,2.0,9.00,7.34,10.87,7.67,1197.400000,199.00,32.40,11.80,6.779215
9838,2018-12-12,2018,12,12,Winter,160.0,1.0,8.61,6.86,11.17,7.97,1106.500000,220.00,36.60,14.00,8.684863
9839,2018-12-12,2018,12,12,Winter,160.0,2.0,8.87,10.33,10.51,7.79,1943.250000,70.60,45.30,5.87,9.031465
9840,2018-12-12,2018,12,12,Winter,180.0,1.0,8.55,9.91,11.07,8.00,1132.900000,84.10,42.30,7.46,7.145236


In [ ]:
# 添加一列Distance的feature
dict_station = {'0.0': 0.0, '10.0': 4.97329372918684, '20.0': 8.67261612026574, '30.0': 14.7019517875022, 
         '40.0': 15.2516905169524, '50.0': 19.3211170838175, '60.0': 26.2875295399737, '70.0': 27.7075206320101, 
         '90.0': 32.0897461330841, '100.0': 36.5634605596958, '110.0': 40.0564570114033, '120.0': 42.8753348278588, '130.0': 47.2185324224328,
         '140.0': 50.3781647129367, '150.0': 54.711049812565, '160.0': 58.9920016079245, '170.0': 65.2622393518111, '180.0': 72.9281362390851}

df_chla_merge['Distance'] = df_chla_merge['Station'].apply(lambda x: dict_station[str(x)])
df_chla_merge['Distance'] = df_chla_merge['Distance'].astype(float32)

# 丢弃冗余时间列
df_chla_merge = df_chla_merge.drop(['Year', 'Month', 'Day'], axis=1)

# 保存到csv中
df_chla_merge.to_csv("General_Chla_data.csv", index=False)

In [ ]:
df_chla_merge

,Date,Season,Station,Depth,YSI_Temp,YSI_Salinity,YSI_DO,YSI_pH,POC,NOx,NH4,PO4,TotalChla,Distance
0,1999-01-05,Winter,0.0,1.0,6.40,0.00,10.57,8.06,1070.700000,1099.89,53.44,16.22,0.219479,0.000000
1,1999-01-05,Winter,0.0,2.0,6.43,0.10,9.96,7.77,1141.666667,1108.73,58.17,20.92,0.798709,0.000000
2,1999-01-05,Winter,20.0,1.0,6.64,0.10,10.74,8.24,1076.300000,1114.91,52.17,19.00,1.002726,8.672616
3,1999-01-05,Winter,20.0,2.0,6.71,0.10,9.94,7.94,1281.066667,1120.90,56.45,22.18,1.210042,8.672616
4,1999-01-05,Winter,30.0,1.0,6.84,0.50,10.65,8.00,750.333333,930.14,82.34,19.32,0.739095,14.701952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9837,2018-12-12,Winter,140.0,2.0,9.00,7.34,10.87,7.67,1197.400000,199.00,32.40,11.80,6.779215,50.378166
9838,2018-12-12,Winter,160.0,1.0,8.61,6.86,11.17,7.97,1106.500000,220.00,36.60,14.00,8.684863,58.992001
9839,2018-12-12,Winter,160.0,2.0,8.87,10.33,10.51,7.79,1943.250000,70.60,45.30,5.87,9.031465,58.992001
9840,2018-12-12,Winter,180.0,1.0,8.55,9.91,11.07,8.00,1132.900000,84.10,42.30,7.46,7.145236,72.928139


## 2. 处理我用的数据

In [ ]:
# 把depth=1留下
df_chla = df_chla_merge[ df_chla_merge['Depth'] == 1 ]

# 增加时间列
df_chla['Year'] = [datetime.strptime(date, '%Y-%m-%d').year for date in df_chla['Date'].astype(str).values.tolist()]
df_chla['Month'] = [datetime.strptime(date, '%Y-%m-%d').month for date in df_chla['Date'].astype(str).values.tolist()]
df_chla['Day'] = [datetime.strptime(date, '%Y-%m-%d').day for date in df_chla['Date'].astype(str).values.tolist()]

# 调整顺序，并去掉深度
order = ['Date', 'Year', 'Month', 'Day', 'Season', 'Station', 'YSI_Temp', 'YSI_Salinity',
         'YSI_DO', 'YSI_pH', 'POC', 'NOx', 'NH4', 'PO4', 'TotalChla', 'Distance']
df_chla = df_chla[order]

df_chla

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

,Date,Year,Month,Day,Season,Station,YSI_Temp,YSI_Salinity,YSI_DO,YSI_pH,POC,NOx,NH4,PO4,TotalChla,Distance
0,1999-01-05,1999,1,5,Winter,0.0,6.40,0.00,10.57,8.06,1070.700000,1099.89,53.44,16.22,0.219479,0.000000
2,1999-01-05,1999,1,5,Winter,20.0,6.64,0.10,10.74,8.24,1076.300000,1114.91,52.17,19.00,1.002726,8.672616
4,1999-01-05,1999,1,5,Winter,30.0,6.84,0.50,10.65,8.00,750.333333,930.14,82.34,19.32,0.739095,14.701952
6,1999-01-05,1999,1,5,Winter,50.0,7.12,0.90,11.37,7.84,902.200000,848.75,55.28,17.67,3.467167,19.321117
8,1999-01-05,1999,1,5,Winter,60.0,7.10,4.60,11.44,7.68,1313.166667,591.49,51.76,16.24,6.831088,26.287529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9832,2018-12-12,2018,12,12,Winter,100.0,8.37,2.72,11.00,7.70,599.850000,391.00,61.60,23.10,3.391824,36.563461
9834,2018-12-12,2018,12,12,Winter,120.0,8.00,2.48,11.59,7.88,1116.850000,387.00,48.60,22.50,4.813551,42.875336
9836,2018-12-12,2018,12,12,Winter,140.0,8.26,3.62,11.54,7.95,1177.600000,371.00,42.30,17.90,7.466879,50.378166
9838,2018-12-12,2018,12,12,Winter,160.0,8.61,6.86,11.17,7.97,1106.500000,220.00,36.60,14.00,8.684863,58.992001


In [ ]:
# 每月选出一天
import dateutil
from datetime import datetime

def select_month_data(df_chla, column_name, type):
  # 定义变量 分类处理数据
  df_group = df_chla.groupby(['Date', 'Year', 'Month', 'Day', 'Season', 'Station'], as_index=False)[column_name].mean()
  df_new = pd.DataFrame(columns=['Date', 'Year', 'Month', 'Day', 'Season', 'Station', column_name])
  station_List = df_group['Station'].unique()

  for station in station_List:
    df_station = df_group[(df_group['Station'] == station)]
    # print(station, ": ", end="")

    # 设置循环变量
    start = datetime.strptime('1999-01', '%Y-%m')
    end = datetime.strptime('2019-01', '%Y-%m')
    while start < end:
      df_temp = df_station[(df_station['Year'] == start.year) & (df_station['Month'] == start.month)]
      # 得到当月所有有测量的日期
      day_array = df_temp['Day'].unique()
      if day_array.size == 0:
        # 9-11: Fall, 12-2: Winter, 3-5: Spring, 6-8: Summer
        season = None
        if start.month in range(6, 9):
          season = 'Summer'
        elif start.month in range(9, 12):
          season = 'Fall'
        elif start.month in range(1, 3) or start.month == 12:
          season = 'Winter'
        elif start.month in range(3, 6):
          season = 'Spring'

        df_new = df_new.append(pd.DataFrame([[start.date(), start.year, start.month, start.day, season, station, np.nan]],
                    columns=['Date', 'Year', 'Month', 'Day', 'Season', 'Station', column_name]))
        # print(start.date(), " ", end="" )
        start += dateutil.relativedelta.relativedelta(months=1)
        continue

      # 添加到新的df中
      if type == 'min':
        df_new = df_new.append(df_temp[df_temp['Day'] == day_array.min()])
      elif type == 'max':
        df_new = df_new.append(df_temp[df_temp['Day'] == day_array.max()])

      start += dateutil.relativedelta.relativedelta(months=1)

    # 打印回车
    # print("")

  return df_new


In [ ]:
def fill_monthly_data(df_new, column_name):
  """
  对于每一个站点，如果缺失一整个月的数据，
  用之前、后 在这个月的数据的平均来补充
  """
  # 缺少测量值的站点名单
  df_blank = df_new[df_new[column_name].isnull()]
  df_fill = df_new.dropna(axis=0)
  station_List = df_blank['Station'].unique()

  for station in station_List:
    # 选出对应的station
    df_station_blank = df_blank[(df_blank['Station'] == station)]
    df_station_fill = df_fill[(df_fill['Station'] == station)]

    month_List = df_station_blank['Month'].unique()
    for month in month_List:
      # 从对应的站点中选出一个月
      df_blank_month = df_station_blank[(df_station_blank['Month'] == month)]
      df_fill_month = df_station_fill[(df_station_fill['Month'] == month)]

      # 计算平均值来填充
      df_blank_month[column_name] = np.mean(df_fill_month[column_name])

      # 添加到总的数据中
      # print(df_blank_month)
      df_fill = df_fill.append(df_blank_month)

  return df_fill


In [ ]:
# 选择第一天作为预测
df_YSI_Temp = select_month_data(df_chla, 'YSI_Temp', 'min')

# 完成插值
df_chla_month = fill_monthly_data(df_YSI_Temp, 'YSI_Temp')

# 按照每一个特征来填充补充
columns = ['YSI_Salinity', 'YSI_DO', 'YSI_pH', 'POC', 'NOx', 'NH4', 'PO4', 'TotalChla']
for col in columns:
  df_temp = select_month_data(df_chla, col, 'min')  # 补充行
  df_temp_filled = fill_monthly_data(df_temp, col)  # 填充值
  df_chla_month = pd.merge(df_chla_month, df_temp_filled, on=['Date', 'Year', 'Month', 'Day', 'Season', 'Station'], how='inner')

# 排序整理
df_chla_month.sort_values(by=['Year', 'Month', 'Station'], axis=0, ascending=True, inplace=True)

df_chla_month

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Date,Year,Month,Day,Season,Station,YSI_Temp,YSI_Salinity,YSI_DO,YSI_pH,POC,NOx,NH4,PO4,TotalChla
0,1999-01-05,1999,1,5,Winter,0.0,6.40,0.00,10.57,8.06,1070.700000,1099.89,53.44,16.22,0.219479
235,1999-01-05,1999,1,5,Winter,20.0,6.64,0.10,10.74,8.24,1076.300000,1114.91,52.17,19.00,1.002726
470,1999-01-05,1999,1,5,Winter,30.0,6.84,0.50,10.65,8.00,750.333333,930.14,82.34,19.32,0.739095
710,1999-01-05,1999,1,5,Winter,50.0,7.12,0.90,11.37,7.84,902.200000,848.75,55.28,17.67,3.467167
950,1999-01-05,1999,1,5,Winter,60.0,7.10,4.60,11.44,7.68,1313.166667,591.49,51.76,16.24,6.831088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1669,2018-12-12,2018,12,12,Winter,100.0,8.37,2.72,11.00,7.70,599.850000,391.00,61.60,23.10,3.391824
1909,2018-12-12,2018,12,12,Winter,120.0,8.00,2.48,11.59,7.88,1116.850000,387.00,48.60,22.50,4.813551
2149,2018-12-12,2018,12,12,Winter,140.0,8.26,3.62,11.54,7.95,1177.600000,371.00,42.30,17.90,7.466879
2389,2018-12-12,2018,12,12,Winter,160.0,8.61,6.86,11.17,7.97,1106.500000,220.00,36.60,14.00,8.684863


In [ ]:
print(df_chla_month.isnull().any())

Date            False
Year            False
Month           False
Day             False
Season          False
Station         False
YSI_Temp        False
YSI_Salinity    False
YSI_DO          False
YSI_pH          False
POC             False
NOx             False
NH4             False
PO4             False
TotalChla       False
dtype: bool


In [ ]:
# 添加一列Distance的feature
dict_station = {'0.0': 0.0, '10.0': 4.97329372918684, '20.0': 8.67261612026574, '30.0': 14.7019517875022, 
         '40.0': 15.2516905169524, '50.0': 19.3211170838175, '60.0': 26.2875295399737, '70.0': 27.7075206320101, 
         '90.0': 32.0897461330841, '100.0': 36.5634605596958, '110.0': 40.0564570114033, '120.0': 42.8753348278588, '130.0': 47.2185324224328,
         '140.0': 50.3781647129367, '150.0': 54.711049812565, '160.0': 58.9920016079245, '170.0': 65.2622393518111, '180.0': 72.9281362390851}

df_chla_month['Distance'] = df_chla_month['Station'].apply(lambda x: dict_station[str(x)])
df_chla_month['Distance'] = df_chla_month['Distance'].astype(float32)

# 丢弃冗余时间列
df_chla_month = df_chla_month.drop(['Year', 'Month', 'Day'], axis=1)

# 保存到csv中
df_chla_month.to_csv("Monthly_Chla_data.csv", index=False)